# Implementation & Evaluation Run RAG (Colab Version)

- **Goal**: For your RAG system, you will need the following three components:
1.	Document & query embedder (can use existing models)
2.	Document retriever (implement sparse, dense and hybrid retrieval)
3.	Document reader (aka. question-answering system) (can use existing models)

- **Inputs**: Data layout expected:
1.  data/corpus/chunks.clean.jsonl
2.   data/test/questions.txt
3.   data/test/reference_answers.json

- **Outputs**:
1. "index" folder
    - bm25.pkl
    - dense_meta.json
    - faiss.index
    - metas.pkl
    - texts.pkl
2. "data/test" folder
    - questions.txt — contains evaluation questions  
    - reference_answers.json — contains reference answers for evaluation  
    - system_output_test.json — generated answers on the test set (Hybrid RRF)

3. "system_outputs" folder.
    - Files are in the form: system_output_1.json

In [2]:
!pip -q install transformers accelerate sentence-transformers faiss-cpu rank-bm25

from pathlib import Path
import json, re, string, collections, numpy as np, torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 84.6 MB/s eta 0:00:00


In [3]:
from google.colab import drive
drive.mount('/content/drive')

import os
print(os.listdir('/content/drive/MyDrive/rag-pgh'))

import faiss, numpy, torch
from rank_bm25 import BM25Okapi
from sentence_transformers import SentenceTransformer
from transformers import AutoTokenizer, AutoModelForCausalLM
print("OK")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
['data', 'scripts', 'seeds.txt', 'rag.py', 'system_outputs', 'index', '__pycache__', 'run_rag_colab.ipynb']
OK


In [6]:
root = Path("/content/drive/MyDrive/rag-pgh")

q_path   = root/"data/test/questions.txt"
ref_path = root/"data/test/reference_answers.json"
out_path = root/"data/test/system_output_test.json"

questions = [ln.strip() for ln in q_path.read_text(encoding="utf-8").splitlines() if ln.strip()]
print("N questions:", len(questions))

N questions: 240


In [7]:
import sys
sys.path.append(str(root))

import rag
from types import SimpleNamespace

In [8]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, AutoModelForCausalLM

class PatchedReader:
    """
    Drop-in replacement for rag.Reader:
      - strict prompt: short span (<=5 words) or 'unknown'
      - supports seq2seq (FLAN/T5) or causal (Mistral/LLaMA)
      - deterministic (no sampling), short outputs
    """
    def __init__(self, model_name: str, max_new_tokens: int = 16, temperature: float = 0.0, include_sources: bool = False):
        self.model_name = model_name
        self.is_seq2seq = any(k in model_name.lower() for k in ["flan","t5"])
        self.tok = AutoTokenizer.from_pretrained(model_name)
        if self.is_seq2seq:
            self.lm = AutoModelForSeq2SeqLM.from_pretrained(model_name, device_map="auto")
        else:
            self.lm = AutoModelForCausalLM.from_pretrained(model_name, device_map="auto")
            if self.tok.pad_token_id is None:
                self.tok.pad_token = self.tok.eos_token
        self.max_new_tokens = max_new_tokens

    def _prompt(self, question: str, passages, max_ctx_chars=1600):
        ctx = ""
        for p in passages[:5]:  # cap to 5 passages
            piece = f"[Passage]\n{p}\n\n"
            if len(ctx) + len(piece) > max_ctx_chars: break
            ctx += piece
        instr = ("Answer with a short span (<=5 words) using ONLY the provided context. "
                 "If the answer is not explicitly present, output exactly: unknown")
        if self.is_seq2seq:
            return f"{instr}\n\nQuestion: {question}\n\nContext:\n{ctx}\n\nAnswer:"
        else:
            return f"[INST]{instr}\n\nQuestion: {question}\n\nContext:\n{ctx}\n\nAnswer:[/INST]"

    def _shorten(self, text: str, max_words=5):
        # first line, strip junk, cap words
        import re
        s = text.splitlines()[0].strip().strip(" .,:;\"'()[]")
        s = re.split(r"(?:^answer\s*:)", s, flags=re.I)[-1].strip()
        words = s.split()
        if not words: return "unknown"
        if len(words) > max_words: s = " ".join(words[:max_words])
        if len(s) > 40 and " " in s: s = " ".join(s.split()[:max_words])
        return s or "unknown"

    def answer(self, question: str, passages, sources=None) -> str:
        prompt = self._prompt(question, passages)
        ids = self.tok(prompt, return_tensors="pt", truncation=True, max_length=2048).to(self.lm.device)
        with torch.no_grad():
            if self.is_seq2seq:
                out = self.lm.generate(**ids, max_new_tokens=self.max_new_tokens, do_sample=False, num_beams=4)
            else:
                out = self.lm.generate(**ids, max_new_tokens=self.max_new_tokens, do_sample=False, temperature=0.0)
        raw = self.tok.decode(out[0], skip_special_tokens=True)
        return self._shorten(raw, max_words=5)

# Monkey-patch for this session:
rag.Reader = PatchedReader
print("✔ Patched rag.Reader (short-span, deterministic).")


✔ Patched rag.Reader (short-span, deterministic).


In [9]:
def nb_index(
    chunks="data/corpus/chunks.clean.jsonl",
    embedder="sentence-transformers/all-MiniLM-L6-v2",
    outdir="index",
    batch=256,
):
    args = SimpleNamespace(chunks=str(root/chunks), embedder=embedder, outdir=str(root/outdir), batch=batch)
    rag.cmd_index(args)
    return {"indexdir": args.outdir}

def nb_answer(
    questions="data/test/questions.txt",
    indexdir="index",
    reader="google/flan-t5-base",     # override TinyLlama
    k=5,
    mode="hybrid",
    fusion="rrf",                     # more robust than weighted
    alpha=0.3,
    pool=40,
    rrfK=60,
    max_new_tokens=16,
    temp=0.0,
    system_out="data/test/system_output_test.json",
    include_sources=False,
):
    args = SimpleNamespace(
        questions=str(root/questions),
        indexdir=str(root/indexdir),
        reader=reader,
        k=k, mode=mode, fusion=fusion, alpha=alpha, pool=pool, rrfK=rrfK,
        max_new_tokens=max_new_tokens, temp=temp,
        system_out=str(root/system_out),
        include_sources=include_sources,
    )
    rag.cmd_answer(args)
    return json.loads(Path(args.system_out).read_text(encoding="utf-8"))

test set

In [ ]:
root = Path("/content/drive/MyDrive/rag-pgh")

# 1) Build / refresh indexes
_ = nb_index(
    chunks="data/corpus/chunks.clean.jsonl",
    embedder="sentence-transformers/all-MiniLM-L6-v2",
    outdir="index",
)

# 2) Generate answers
# HYBRID (RRF)
_ = nb_answer(
    questions="data/test/questions.txt",
    indexdir="index",
    reader="google/flan-t5-base",
    mode="hybrid",
    fusion="rrf",
    k=5, pool=40,
    max_new_tokens=16, temp=0.0,
    system_out="data/test/system_output_test.json",  # main hybrid file
    include_sources=False,
)

# SPARSE (BM25)
_ = nb_answer(
    questions="data/test/questions.txt",
    indexdir="index",
    reader="google/flan-t5-base",
    mode="sparse",
    fusion="rrf",
    k=5, pool=40,
    max_new_tokens=16, temp=0.0,
    system_out="data/test/system_output_sparse.json",
    include_sources=False,
)

# DENSE (FAISS)
_ = nb_answer(
    questions="data/test/questions.txt",
    indexdir="index",
    reader="google/flan-t5-base",
    mode="dense",
    fusion="rrf",
    k=5, pool=40,
    max_new_tokens=16, temp=0.0,
    system_out="data/test/system_output_dense.json",
    include_sources=False,
)

print("Wrote:")
print("- data/test/system_output_test.json     (hybrid)")
print("- data/test/system_output_sparse.json   (sparse)")
print("- data/test/system_output_dense.json    (dense)")

In [23]:
import json
from pathlib import Path

root = Path("/content/drive/MyDrive/rag-pgh")
q_path   = root/"data/test/questions.txt"
ref_path = root/"data/test/reference_answers.json"
sys_path = root/"data/test/system_output_test.json"

questions = [ln.strip() for ln in q_path.read_text(encoding="utf-8").splitlines() if ln.strip()]
pred = json.loads(sys_path.read_text(encoding="utf-8"))
refs = json.loads(ref_path.read_text(encoding="utf-8"))

for i, q in enumerate(questions, start=1):
    k = str(i)
    print(f"{i:>3}. Q: {q}\n     A: {pred.get(k,'')}\n     G: {refs.get(k,'')}\n")

  1. Q: 1. When was Carnegie Mellon University founded?
     A: spring of 2006
     G: 1900

  2. Q: 2. Who founded Carnegie Mellon University?
     A: the world’s first machine learning
     G: Andrew Carnegie

  3. Q: 3. What was the original name of Carnegie Mellon University?
     A: unknown
     G: Carnegie Technical Schools

  4. Q: 4. What academic fields is Carnegie Mellon University best known for?
     A: artificial intelligence, cybersecurity, information systems,
     G: computer science; robotics; artificial intelligence; engineering

  5. Q: 5. What are the major colleges or schools within Carnegie Mellon University?
     A: Carnegie Mellon University is tied
     G: College of Engineering; School of Computer Science; Dietrich College of Humanities and Social Sciences; Tepper School of Business; Heinz College; Mellon College of Science; School of Architecture

  6. Q: 6. Where is Carnegie Mellon University located?
     A: United States
     G: Pittsburgh, Pennsylvania

 

unseen data

In [14]:
import pandas as pd
from pathlib import Path

# paths
root = Path("/content/drive/MyDrive/rag-pgh")
csv_path = root / "system_outputs/test_set_Day1.csv"
out_q_path = root / "system_outputs/day1_questions.txt"

# 1) Load CSV safely with pandas
try:
    df = pd.read_csv(csv_path)
except Exception as e:
    print(" pandas read_csv failed, retrying with python engine:", e)
    df = pd.read_csv(csv_path, engine="python")

print("Columns:", list(df.columns))

# 2) Automatically detect which column has the questions
# Preference order by common names
candidate_cols = [c for c in df.columns if c.strip()]
question_col = None
for cand in ["question", "Question", "questions", "Questions", "text", "Text", "prompt"]:
    if cand in df.columns:
        question_col = cand
        break

# If none matched, pick the first column with non-empty text values
if question_col is None:
    non_empty_counts = df.notna().sum().to_dict()
    question_col = max(non_empty_counts, key=non_empty_counts.get)

print(f"Detected question column: {question_col}")

# 3) Clean and extract question list
questions = (
    df[question_col]
    .astype(str)
    .apply(lambda x: x.strip())
    .loc[lambda s: s != ""]
    .tolist()
)

questions = [q for q in questions if q.lower() not in {"nan", "none", "null"}]

# 4) Save as text
out_q_path.write_text("\n".join(questions) + "\n", encoding="utf-8")
print(f" Saved {len(questions)} questions to {out_q_path}")

# 5) Preview a few
for i, q in enumerate(questions[:5], start=1):
    print(f"{i:>3}. {q}")


Columns: ['Question']
Detected question column: Question
 Saved 400 questions to /content/drive/MyDrive/rag-pgh/system_outputs/day1_questions.txt
  1. What year did Pittsburgh suffer its worst flood?
  2. When did the Carnegie-Mellon merger occur?
  3. What Kennywood food is most iconic?
  4. How many schools and colleges does CMU have?
  5. What former church is a music venue?


In [15]:
answers = nb_answer(
    questions=str(out_q_path.relative_to(root)),
    indexdir="index",
    reader="google/flan-t5-base",
    mode="hybrid",
    fusion="rrf",
    k=5, pool=40,
    max_new_tokens=16, temp=0.0,
    system_out="system_outputs/system_output_1.json",
    include_sources=False,
)
print(" Answers written to system_outputs/system_output_1.json")


Loaded index with 3197 chunks


Answering:   0%|          | 1/400 [00:01<10:14,  1.54s/it]

Q1 sources:
  [1] Public Safety - Pittsburgh, PA — https://www.pittsburghpa.gov/Safety/Alerts/Public-Safety-Press-Release-Archive
  [2] Public Safety - Pittsburgh, PA — https://www.pittsburghpa.gov/Safety/Alerts/Public-Safety-Press-Release-Archive
  [3] Public Safety - Pittsburgh, PA — https://www.pittsburghpa.gov/Safety/Alerts/Public-Safety-Press-Release-Archive
  [4] Public Safety - Pittsburgh, PA — https://www.pittsburghpa.gov/Safety/Alerts/Public-Safety-Press-Release-Archive
  [5] Public Safety - Pittsburgh, PA — https://www.pittsburghpa.gov/Safety/Alerts/Public-Safety-Press-Release-Archive
Q2 sources:
  [1] Legal | Carnegie Mellon University — https://www.cmu.edu/legal/
  [2] Legal | Carnegie Mellon University — https://www.cmu.edu/legal/
  [3] Legal | Carnegie Mellon University — https://www.cmu.edu/legal/
  [4] Regional Impact | Carnegie Mellon University — https://www.cmu.edu/regional-impact
  [5] Carnegie Mellon Named a Top 20 US University — https://www.cmu.edu/news/stories/a

Answering:   1%|          | 3/400 [00:01<03:12,  2.06it/s]

Q3 sources:
  [1] Kennywood | Visit Pittsburgh — https://www.visitpittsburgh.com/directory/kennywood/
  [2] Pittsburgh Food Festivals - Greek - Oktoberfest - Apples - Maple Syrup — https://www.visitpittsburgh.com/events-festivals/food-festivals/
  [3] Public Safety - Pittsburgh, PA — https://www.pittsburghpa.gov/Safety/Alerts/Public-Safety-Press-Release-Archive
  [4] Dining Partners — https://pittsburghsymphony.org/pso_home/web/give-landing/corporate-partnerships/dining-partners
  [5] Pistons in the Park Car Cruise — https://www.pittsburghpa.gov/Recreation-Events/Events/Pistons-in-the-Park-Car-Cruise
Q4 sources:
  [1] Giving Opportunities | Engage with CMU — https://www.cmu.edu/engage/give/opportunities/index.html
  [2] CMU Undergraduates Take on Cancer Research — https://www.cmu.edu/news/stories/archives/2025/may/cmu-undergraduates-take-on-cancer-research
  [3] CMU Celebrates 2025 Fulbright Recipients — https://www.cmu.edu/news/stories/archives/2025/august/cmu-celebrates-2025-fulbrigh

...

Q397 sources:
  [1] What's Happening - Downtown Pittsburgh — https://downtownpittsburgh.com/events/?n=10&d=22&y=2025
  [2] What's Happening - Downtown Pittsburgh — https://downtownpittsburgh.com/events/?n=10&d=29&y=2025
  [3] What's Happening - Downtown Pittsburgh — https://downtownpittsburgh.com/events/?n=11&d=2&y=2025
  [4] What's Happening - Downtown Pittsburgh — https://downtownpittsburgh.com/events/?n=10&d=4&y=2025
  [5] What's Happening - Downtown Pittsburgh — https://downtownpittsburgh.com/events/?n=11&y=2025
Q398 sources:
  [1] Event - Downtown Pittsburgh — https://downtownpittsburgh.com/events/event/?id=63876
  [2] A New Theater, a Significant Sketch, and a Giant Duck: Three Big Moments in Trust History — https://trustarts.org/blog/a-new-theater-a-significant-sketch-and-a-giant-duck-three-big-moments-in-trust-history
  [3] Blog — https://trustarts.org/blog
  [4] Event - Downtown Pittsburgh — https://downtownpittsburgh.com/events/event/?id=62499
  [5] What's Happening - Downtow

Answering: 100%|█████████▉| 399/400 [01:40<00:00,  3.55it/s]

Q399 sources:
  [1] Family Weekend - Family Engagement - Carnegie Mellon University — https://www.cmu.edu/family/events/family-weekend/
  [2] Family Weekend - Family Engagement - Carnegie Mellon University — https://www.cmu.edu/family/events/family-weekend/
  [3] What's Happening - Downtown Pittsburgh — https://downtownpittsburgh.com/events/?n=10&d=22&y=2025
  [4] Event - Downtown Pittsburgh — https://downtownpittsburgh.com/events/event/?id=63965
  [5] What's Happening - Downtown Pittsburgh — https://downtownpittsburgh.com/events/?n=10&d=12&y=2025
Q400 sources:
  [1] Event - Downtown Pittsburgh — https://downtownpittsburgh.com/events/event/?id=63881
  [2] Event - Downtown Pittsburgh — https://downtownpittsburgh.com/events/event/?id=63879
  [3] What's Happening - Downtown Pittsburgh — https://downtownpittsburgh.com/events/?n=10&d=10&y=2025
  [4] What's Happening - Downtown Pittsburgh — https://downtownpittsburgh.com/events/?n=12&d=1&y=2025
  [5] Blog — https://trustarts.org/blog


Answering: 100%|██████████| 400/400 [01:40<00:00,  3.98it/s]

Wrote /content/drive/MyDrive/rag-pgh/system_outputs/system_output_1.json
 Answers written to system_outputs/system_output_1.json


In [16]:
questions = [ln.strip() for ln in out_q_path.read_text(encoding="utf-8").splitlines() if ln.strip()]
pred = json.loads(out_json.read_text(encoding="utf-8"))

print("\n--- Preview ---")
for i, q in enumerate(questions, start=1):
    k = str(i)
    print(f"{i:>3}. Q: {q}\n     A: {pred.get(k, '')}\n")


--- Preview ---
  1. Q: What year did Pittsburgh suffer its worst flood?
     A: 2011

  2. Q: When did the Carnegie-Mellon merger occur?
     A: unknown

  3. Q: What Kennywood food is most iconic?
     A: apple, maple and whiskey

  4. Q: How many schools and colleges does CMU have?
     A: seven

  5. Q: What former church is a music venue?
     A: unanswerable

  6. Q: What year does Pittsburgh host NFL Draft?
     A: 2026

  7. Q: What is the area in square miles of Squirrel Hill as stated by the Squirrel Hill Historical Society?
     A: unknown

  8. Q: In what year did Carnegie Institute of Technology merge with the Mellon Institute to form Carnegie Mellon University?
     A: unanswerable

  9. Q: What year was the Whiskey Rebellion?
     A: unanswerable

 10. Q: In what year was the Frick Environmental Center opened to the public?
     A: 1989

 11. Q: When is fourth quarter Payroll Tax due in Pittsburgh?
     A: February 28th

 12. Q: What is Pittsburgh's business district ca

evaluation

In [21]:
# =========================
# RAG Evaluation
# =========================

import re, string, collections, math

# --- Paths ---
root = Path("/content/drive/MyDrive/rag-pgh")
q_path   = root/"data/test/questions.txt"
ref_path = root/"data/test/reference_answers.json"
paths = {
    "sparse": root/"data/test/system_output_sparse.json",
    "dense":  root/"data/test/system_output_dense.json",
    "hybrid": root/"data/test/system_output_test.json",
}

# --- Normalization + F1 helpers ---
def _normalize_text(s: str) -> str:
    s = s.lower().strip()
    s = s.translate(str.maketrans("", "", string.punctuation))
    return " ".join(s.split())

def _f1_single(pred: str, gold: str) -> float:
    ptoks = _normalize_text(pred).split()
    gtoks = _normalize_text(gold).split()
    if not ptoks and not gtoks: return 1.0
    if not ptoks or not gtoks:  return 0.0
    common = collections.Counter(ptoks) & collections.Counter(gtoks)
    num_same = sum(common.values())
    if num_same == 0: return 0.0
    precision = num_same / len(ptoks)
    recall    = num_same / len(gtoks)
    return 2 * precision * recall / (precision + recall)

def _best_em_f1(pred: str, gold_variants: str):
    variants = [g.strip() for g in gold_variants.split(";") if g.strip()]
    if not variants: return 0, 0.0
    pnorm = _normalize_text(pred)
    best_em, best_f1 = 0, 0.0
    for g in variants:
        gnorm = _normalize_text(g)
        em = int(pnorm == gnorm)
        f1 = _f1_single(pred, g)
        best_em = max(best_em, em)
        best_f1 = max(best_f1, f1)
    return best_em, best_f1

# --- Buckets ---
_MONTH = r"(jan|feb|mar|apr|may|jun|jul|aug|sep|oct|nov|dec)"
_DATE  = rf"(\b{_MONTH}\b|\b{_MONTH}[a-z]*\b|\b\d{{1,2}}/\d{{1,2}}\b|\b\d{{4}}\b)"
_BUCKETS = {
    "when/date":   re.compile(rf"\b(when|date|year|on)\b|{_DATE}", re.I),
    "who/person":  re.compile(r"\b(who|artist|singer|performer|mayor)\b", re.I),
    "where/venue": re.compile(r"\b(where|located|venue)\b|\b(Hall|Center|Park|Museum|Arena|Theater|Theatre|Stadium)\b", re.I),
    "events":      re.compile(r"\b(event|concert|festival|show)\b|\bPPG\b", re.I),
    "cmu-facts":   re.compile(r"\b(cmu|carnegie mellon)\b", re.I),
    "pgh-facts":   re.compile(r"\b(pittsburgh)\b", re.I),
}
_BUCKET_ORDER = ["when/date", "who/person", "where/venue", "events", "cmu-facts", "pgh-facts"]

def _bucket(q: str) -> str:
    for name in _BUCKET_ORDER:
        if _BUCKETS[name].search(q):
            return name
    return "other"

# --- Evaluation function ---
def evaluate_one(sys_path: Path):
    questions = [ln.strip() for ln in q_path.read_text(encoding="utf-8").splitlines() if ln.strip()]
    gold = json.loads(ref_path.read_text(encoding="utf-8"))
    pred = json.loads(sys_path.read_text(encoding="utf-8"))

    exp = [str(i+1) for i in range(len(questions))]
    assert list(gold.keys()) == exp, "reference_answers.json keys must be '1'..'N'"
    assert list(pred.keys()) == exp, f"{sys_path.name} keys must be '1'..'N'"

    perq = []
    ems, f1s = [], []
    nonempty = nonunknown = 0

    for i, q in enumerate(questions, start=1):
        k = str(i)
        p = pred.get(k, "")
        g = gold.get(k, "")
        em, f1 = _best_em_f1(p, g)
        ems.append(em); f1s.append(f1)
        nonempty   += int(bool(p.strip()))
        nonunknown += int(p.strip().lower() not in ("unknown", ""))

        perq.append({
            "id": i,
            "bucket": _bucket(q),
            "question": q,
            "pred": p,
            "gold": g,
            "EM": em,
            "F1": f1
        })

    EM = sum(ems)/len(ems)
    F1 = sum(f1s)/len(f1s)

    # per-bucket breakdown
    agg = collections.defaultdict(lambda: {"n":0,"EM":0.0,"F1":0.0})
    for r in perq:
        b = r["bucket"]; agg[b]["n"] += 1; agg[b]["EM"] += r["EM"]; agg[b]["F1"] += r["F1"]
    buckets = {b: {"n":v["n"], "EM": v["EM"]/v["n"] if v["n"] else 0.0,
                        "F1": v["F1"]/v["n"] if v["n"] else 0.0}
               for b,v in agg.items()}

    return {
        "N": len(questions),
        "EM": EM, "F1": F1,
        "nonempty_rate": nonempty/len(questions),
        "nonunknown_rate": nonunknown/len(questions),
        "per_question": perq,
        "per_bucket": buckets
    }

def print_summary(name: str, r: dict):
    print(f"== {name} ==")
    print(f"N={r['N']}  EM={r['EM']:.3f}  F1={r['F1']:.3f}  non-empty={r['nonempty_rate']:.3f}  non-'unknown'={r['nonunknown_rate']:.3f}")
    if r["per_bucket"]:
        print("Per-bucket:")
        for b, v in sorted(r["per_bucket"].items()):
            print(f"  {b:12s}  n={v['n']:>3}  EM={v['EM']:.3f}  F1={v['F1']:.3f}")
    print()

# --- Run for all retrieval types ---
results = {}
for name, p in paths.items():
    if p.exists():
        results[name] = evaluate_one(p)
        print_summary(name, results[name])
    else:
        print(f"[warn] Missing file for {name}: {p}")

# --- Simple ablation summary ---
print("Ablation (EM/F1):")
for name in ["sparse", "dense", "hybrid"]:
    if name in results:
        print(f"  {name:6s}  EM={results[name]['EM']:.3f}   F1={results[name]['F1']:.3f}")


== sparse ==
N=240  EM=0.121  F1=0.238  non-empty=1.000  non-'unknown'=0.887
Per-bucket:
  cmu-facts     n= 39  EM=0.231  F1=0.323
  events        n= 19  EM=0.053  F1=0.147
  other         n= 22  EM=0.136  F1=0.240
  pgh-facts     n= 68  EM=0.044  F1=0.135
  when/date     n= 31  EM=0.129  F1=0.246
  where/venue   n= 57  EM=0.123  F1=0.295
  who/person    n=  4  EM=0.500  F1=0.714

== dense ==
N=240  EM=0.146  F1=0.243  non-empty=1.000  non-'unknown'=0.904
Per-bucket:
  cmu-facts     n= 39  EM=0.231  F1=0.332
  events        n= 19  EM=0.105  F1=0.190
  other         n= 22  EM=0.091  F1=0.200
  pgh-facts     n= 68  EM=0.074  F1=0.141
  when/date     n= 31  EM=0.129  F1=0.250
  where/venue   n= 57  EM=0.175  F1=0.285
  who/person    n=  4  EM=0.750  F1=0.964

== hybrid ==
N=240  EM=0.146  F1=0.259  non-empty=1.000  non-'unknown'=0.908
Per-bucket:
  cmu-facts     n= 39  EM=0.205  F1=0.336
  events        n= 19  EM=0.053  F1=0.153
  other         n= 22  EM=0.136  F1=0.249
  pgh-facts     n=

In [22]:
def paired_t_test_F1(resA, resB):
    A = [r["F1"] for r in resA["per_question"]]
    B = [r["F1"] for r in resB["per_question"]]
    assert len(A)==len(B)
    diffs = [a-b for a,b in zip(A,B)]
    n = len(diffs)
    mean_d = sum(diffs)/n
    var_d  = sum((d-mean_d)**2 for d in diffs)/(n-1) if n>1 else 0.0
    se     = (var_d/n)**0.5 if n>0 else 1.0
    t = mean_d/se if se>0 else 0.0
    # Normal approx to p-value (fine at N>=30)
    from math import erf, sqrt
    p = 2*(1 - 0.5*(1+erf(abs(t)/sqrt(2))))
    return t, p

def mcnemar_EM(resA, resB):
    A = [r["EM"] for r in resA["per_question"]]
    B = [r["EM"] for r in resB["per_question"]]
    b01 = sum(1 for a,b in zip(A,B) if a==0 and b==1)  # B correct, A wrong
    b10 = sum(1 for a,b in zip(A,B) if a==1 and b==0)  # A correct, B wrong
    # Continuity-corrected chi^2
    chi2 = ((abs(b01-b10)-1)**2) / (b01+b10) if (b01+b10)>0 else 0.0
    from math import erf, sqrt
    p = 1 - erf((chi2/2)**0.5)   # chi^2(1) approx
    return b01, b10, chi2, p

if "hybrid" in results and "sparse" in results:
    t,p = paired_t_test_F1(results["hybrid"], results["sparse"])
    b01,b10,chi2,pm = mcnemar_EM(results["hybrid"], results["sparse"])
    print("\nHybrid vs Sparse:")
    print(f"  paired t-test on F1: t={t:.3f}, p≈{p:.4f}")
    print(f"  McNemar on EM: b01={b01}, b10={b10}, chi2={chi2:.3f}, p≈{pm:.4f}")

if "hybrid" in results and "dense" in results:
    t,p = paired_t_test_F1(results["hybrid"], results["dense"])
    b01,b10,chi2,pm = mcnemar_EM(results["hybrid"], results["dense"])
    print("\nHybrid vs Dense:")
    print(f"  paired t-test on F1: t={t:.3f}, p≈{p:.4f}")
    print(f"  McNemar on EM: b01={b01}, b10={b10}, chi2={chi2:.3f}, p≈{pm:.4f}")



Hybrid vs Sparse:
  paired t-test on F1: t=0.964, p≈0.3350
  McNemar on EM: b01=7, b10=13, chi2=1.250, p≈0.2636

Hybrid vs Dense:
  paired t-test on F1: t=0.873, p≈0.3829
  McNemar on EM: b01=8, b10=8, chi2=0.062, p≈0.8026
